# 0.0 Objetivo 

# 1.0 Importando as bibliotecas

In [1]:
import pandas         as pd
import numpy          as np
import seaborn        as sns
import plotly.express as px
import ipywidgets     as widgets

from ipywidgets            import fixed
from matplotlib            import gridspec
from matplotlib            import pyplot as plt
from geopy.geocoders       import Nominatim
from IPython.core.display  import HTML
from IPython.display       import Image

## 1.1 - Funções

In [2]:
# Criação de Botões
def bt_intslider(Val,Min,Max,desc=''):
    
    bt = widgets.IntSlider(value = Val, 
                           min = Min,
                           max = Max, 
                           step = 1,
                           description=desc,
                           disable=False,
                           orientation='horizontal',
                           style = {'description_width': 'initial'})
    return bt    

# Criação de Botões
def bt_dropdown(data,x='',desc=''):
    
    bt1 = widgets.Dropdown(
    options=data[x].sort_values().unique().tolist(),
    description= desc,
    disabled=False,
    style={'description_width': 'initial'})
    
    return bt1   



# Estatística Descritiva
def num_metricas(num_attributes):
  #Central tendencian - mean, median

  ct1 = pd.DataFrame( num_attributes.apply(np.mean)).T
  ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

  #Dispersion - std, min, max, range, skew, kurtosis

  d1 = pd.DataFrame(num_attributes.apply(np.std)).T
  d2 = pd.DataFrame(num_attributes.apply(min)).T
  d3 = pd.DataFrame(num_attributes.apply(max)).T
  d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min() )).T
  d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew() )).T
  d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis() )).T

  #Concatenate
  metrics = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
  metrics.columns = ['attributes','min', 'max', 'range', 'mean', 'median', 'std', 'skew','kurtosis']
  return metrics


def jupyter_settings():
    
    %matplotlib inline 
    #%pylab inline
    
    plt.style.use('bmh')
    plt.rcParams['figure.figsize']=[20,10]
    plt.rcParams['font.size']=10
    
    display( HTML('<style>.container {width:100% !important; }</style>'))
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option('display.expand_frame_repr',False )
    pd.set_option('display.float_format',lambda x: '%.2f' % x)
    

    sns.set()
    


jupyter_settings()

## 1.2 - Carregando os dados

In [3]:
df_raw = pd.read_csv('../data/kc_house_data.csv')
df = df_raw.copy()
df.sample(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
12514,3332500636,20140605T000000,356000.00,2,1.00,920,4095,1.00,0,0,4,6,920,0,1914,0,98118,47.55,-122.28,1460,4945
19929,8024200685,20140520T000000,440000.00,3,1.50,1270,1443,3.00,0,0,3,8,1270,0,2007,0,98115,47.70,-122.32,1270,1413
10980,5729000070,20150128T000000,545000.00,4,2.00,5461,22880,1.00,0,0,4,9,3265,2196,1964,0,98032,47.36,-122.29,1940,10995
4888,7140200380,20141030T000000,275000.00,3,2.00,1910,8050,1.00,0,0,4,7,1000,910,1980,0,98030,47.37,-122.17,1780,7344
7872,1454100650,20140804T000000,942000.00,4,2.75,3160,37200,2.00,0,3,3,8,2310,850,1939,0,98125,47.72,-122.28,3130,20000


# 2.0 Descrição dos Dados

## 2.1 Renomear as Colunas

In [4]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

Neste caso não vai ser necessário 

## 2.2 Dimensão dos Dados

In [5]:
print('Número de linhas: {}'.format(df.shape[0]))
print('Número de colunas: {}'.format(df.shape[1]))

Número de linhas: 21613
Número de colunas: 21


## 2.3 Tipos dos Dados

In [6]:
df.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

### 1.3.1 Mudança do tipo da variável

Primeiramente vamos colocar a variável "date" que está como object para o tipo datetime.

In [7]:
# mudando o formato da data
df['year'] = pd.to_datetime(df['date']).dt.strftime('%Y')
df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
df['year_week'] = pd.to_datetime(df['date']).dt.strftime('%Y-%U')

## 2.4 Valores nulos no Dataset

In [8]:
df.isnull().sum()

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
year             0
year_week        0
dtype: int64

Pode - se perceber que o dataset não tem valores nulos.

## 2.5 Estatística Descritiva

In [9]:
# Separando as variáveis em categórica e numérica
num_atributos = df.select_dtypes(include = ['int64', 'float64'])
cat_atributos= df.select_dtypes(include = ['object'])

### 1.5.1 Atributos Numéricos

In [10]:
num_metricas(num_atributos)

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,id,1000102.00,9900000190.00,9899000088.00,4580301520.86,3904930410.00,2876499023.43,0.24,-1.26
1,price,75000.00,7700000.00,7625000.00,540088.14,450000.00,367118.70,4.02,34.59
2,bedrooms,0.00,33.00,33.00,3.37,3.00,0.93,1.97,49.06
3,bathrooms,0.00,8.00,8.00,2.11,2.25,0.77,0.51,1.28
4,sqft_living,290.00,13540.00,13250.00,2079.90,1910.00,918.42,1.47,5.24
5,sqft_lot,520.00,1651359.00,1650839.00,15106.97,7618.00,41419.55,13.06,285.08
6,floors,1.00,3.50,2.50,1.49,1.50,0.54,0.62,-0.48
7,waterfront,0.00,1.00,1.00,0.01,0.00,0.09,11.39,127.63
8,view,0.00,4.00,4.00,0.23,0.00,0.77,3.40,10.89
9,condition,1.00,5.00,4.00,3.41,3.00,0.65,1.03,0.53


### 1.5.2 Atributos Categóricos 

In [11]:
cat_atributos.columns

Index(['date', 'year', 'year_week'], dtype='object')

Dataset não apresenta variáveis categóricas



## 2.6 Criação de Variáveis

### 2.6.1 Condition house

In [12]:
#Condition house
df['condition_type']= 'NA'
df['condition_type']= df['condition'].apply(lambda x: 'bad'  if x <= 2 else 
                                            'regular' if (x >= 3) & (x <= 4) 
                                            else 'good')  


### 2.6.2 Level

- Nível 0 -> Preço entre 0 e 321.950
- Nível 1 -> Preço entre 321.950 e 450.000
- Nível 2 -> Preço entre 450.000 e 645.000
- Nível 3 -> Acima de 645.000

In [13]:
df['level']= 'NA'
df['level']= df['price'].apply(lambda x: 0  if x < 321950 else 
                                              1  if (x >= 321950) & (x < 450000) else 
                                              2  if (x >= 450000) & (x < 645000) else 3)  
df['level'] = df['level'].astype(int)


### 2.6.3 Size

- Size 0 -> Tamanho entre 0 e 1427 sqft
- Size 1 -> Tamanho entre 1427 e 1910 sqft
- Size 2 -> Tamanho entre 1910 e 2550 sqft
- Size 3 -> Tamanho acima de 2550 sqft

In [14]:
df['size']= 'NA'
df['size']= df['sqft_living'].apply(lambda x: 0  if x < 1427 else 
                                              1  if (x >= 1427) & (x < 1910) else 
                                              2  if (x >= 1910) & (x < 2550) else 3)  
df['size'] = df['size'].astype(int)

### 2.6.4 Waterfront

In [15]:
df['is_waterfront'] = df['waterfront'].apply( lambda x: 'yes' if x == 1 else 'no' )

# 3.0 - Qual a média do preço de compra dos imóveis por "Nível"?


In [16]:
aux = [] 
for i in range(4): 
    m = (df['level'] == i).sum()/(df.shape[0])
    aux.append(m)
    media_level= pd.DataFrame(aux).reset_index()
    media_level.columns = ['Level','Mean']

media_level
    


,Level,Mean
0,0,0.25
1,1,0.24
2,2,0.25
3,3,0.25


# 4.0 - Qual a média do tamanho da sala de estar dos imóveis por "Size"?


In [17]:
aux01 = [] 
for i in range(4): 
    m = (df['size'] == i).sum()/(df.shape[0])
    aux01.append(m)
    media_size= pd.DataFrame(aux01).reset_index()
    media_size.columns = ['Size','Mean']

media_size

,Size,Mean
0,0,0.25
1,1,0.25
2,2,0.25
3,3,0.25


# 5.0 - Adicione as seguintes informações ao conjunto de dados original:


- PlaceID: identificação da localização.
- OSM Type: Open Street Map Type
- Country: Nome do País
- Country Code: Código do país

In [28]:
df4 = df.copy()

In [29]:
lista = ['country','state','city','neighbourhood','road','house_number']

# Cria linhas vazias
df4['country'] = 'NA'
df4['state'] = 'NA'
df4['city'] = 'NA'
df4['neighbourhood'] = 'NA'
df4['road'] = 'NA'
df4['house_number'] = 'NA'

# Inicializar API
geolocator = Nominatim(user_agent='geoapiExercises')

for i in range(len(df4)):
    
    print('Loop:{} / {}'.format(i,len(df4)))
    
    query = str(df4.loc[i,'lat']) + ','+ str(df4.loc[i,'long'])
    
    # Requisição na API 
    response = geolocator.reverse(query)
    
    # Verificar se tem as informações no json
    for j in lista:
      
      if 'j' in response.raw['address']:
        df4.loc[i,'j'] = response.raw['address']['country']

Loop:0 / 21613
Loop:1 / 21613
Loop:2 / 21613
Loop:3 / 21613
Loop:4 / 21613
Loop:5 / 21613
Loop:6 / 21613
Loop:7 / 21613
Loop:8 / 21613
Loop:9 / 21613
Loop:10 / 21613
Loop:11 / 21613
Loop:12 / 21613
Loop:13 / 21613
Loop:14 / 21613
Loop:15 / 21613
Loop:16 / 21613
Loop:17 / 21613
Loop:18 / 21613
Loop:19 / 21613
Loop:20 / 21613
Loop:21 / 21613
Loop:22 / 21613
Loop:23 / 21613
Loop:24 / 21613


KeyboardInterrupt: 

Este exercício já foi realizado e está no notebook do Exercício 04. Foi salvo um dataset com as informações coletadas na API.

# 6.0 - Adicione os seguintes filtros no mapa:


- Tamanho mínimo da área da sala estar.
- Número mínimo de banheiros.
- Valor máximo do Preço.
- Tamanho máximo da área do porão.
- Filtro das condições do imóvel.
- Filtro por ano de construção.

In [20]:
#Botões interativos

year_built = bt_intslider(1900,1900,2015, desc='Ano de Construção')
area_basement = bt_intslider(0,0,4820, desc='Área do Porão')
room_size = bt_intslider(290,290,13540, desc='Sala de Estar')
limit_price = bt_intslider(540000,75000,77000000, desc='Preço Máximo')

bathroom_qtd = bt_dropdown(df,x='bathrooms',desc='Banheiros')
type_house = bt_dropdown(df,x='condition_type',desc='Condição do imóvel')



In [21]:
def update_map( df,bathroom, condition, year, room, limit, basement):
    
    #Definindo as colunas para o mapa
    houses = df[(df['sqft_living'] >= room) &
                (df['bathrooms'] >= bathroom) &
                (df['price'] <= limit) & 
                (df['sqft_basement'] >= basement) & 
                (df['condition_type'] >= condition) & 
                (df['yr_built'] >= year)][['sqft_living','bathrooms','sqft_basement','condition','yr_built','id','price','lat','long','level']]

    #Criando mapa
    maps_price = px.scatter_mapbox(data_frame= houses,
                lat='lat',
                lon='long',
                hover_name='id',
                color='level',                  
                size='price',                  
                color_discrete_sequence=['darkblue'],
                color_continuous_scale=px.colors.sequential.Viridis,
                size_max=30,                  
                zoom=8,
                mapbox_style='open-street-map',
                height=1000) 

    #Definindo as margens do mapa
    maps_price.update_layout(margin={'r':0, 't':0, 'l':0, 'b':0})
    maps_price.show()



In [22]:
widgets.interactive( update_map, df=fixed(df), bathroom=bathroom_qtd,condition=type_house, year=year_built, room=room_size, limit=limit_price, basement=area_basement)

interactive(children=(Dropdown(description='Banheiros', options=(0.0, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.…

# 7.0 - Adicione os seguintes filtros no Dashboard:
- Filtro por data disponível para compra.
- Filtro por ano de renovação.
- Filtro se possui vista para a água ou não

In [23]:
df1 = df.copy()

In [24]:
# Criando o botão para filtrar os dados

date_limit = bt_dropdown(df1,x='date',desc='Disponível para Compra')
date_renovated = bt_dropdown(df1,x='yr_renovated',desc='Ano de Renovação')
waterfront_bar = bt_dropdown(df1,x='is_waterfront',desc='Vista para o mar')

In [25]:
def update_dash( data, date, renovated, waterfront):
    # Filtrar data
    df = data[(data['date'] <= date) & 
              (data['yr_renovated'] >= renovated) & 
              (data['is_waterfront'] == waterfront)]


    fig = plt.figure( figsize=(21,12))
    specs = gridspec.GridSpec( ncols=2, nrows=2, figure=fig)
    
    ax1 = fig.add_subplot( specs[0, :]) # Primeira linha
    ax2 = fig.add_subplot( specs[1, 0]) # Segunda Linha e Primeira coluna
    ax3 = fig.add_subplot( specs[1, 1]) # Segunda Linha e Segunda coluna
    
    by_year = data[['id', 'year']].groupby('year').sum().reset_index()
    ax1.bar( by_year['year'], by_year['id'])
    ax1.set_title( 'Quantidade de imóvel disponível para compra', fontsize=16)
    
    by_day = data[['id', 'date']].groupby( 'date').mean().reset_index()
    ax2.plot( by_day['date'], by_day['id'])
    ax2.set_title( 'Preço do imóvel por dia', fontsize=16)
    
    by_week_of_year = data[['id', 'year_week']].groupby( 'year_week').mean().reset_index()
    ax3.bar( by_week_of_year['year_week'], by_week_of_year['id'])
    ax3.set_title( 'Preço do imóvel por semana do ano', fontsize=16)
    plt.xticks( rotation=50);


In [26]:
widgets.interactive( update_dash, data=fixed(df1), date=date_limit, renovated=date_renovated, waterfront=waterfront_bar)

interactive(children=(Dropdown(description='Disponível para Compra', options=('2014-05-02', '2014-05-03', '201…